So we've started some of our journey onto bayesian probabilities.

# Summary of Bayesian

# Simple Probability Problem Jar of Marbles

Let's say I have one jar of marbles. The it has 70 red marbles and 30 blue marbles. I pull marbles out and get the sequence Red, Blue, Blue, Red. What is the probability that I pulled this sequence?

# Slightly More Challenging Probability Problem Jar of Marbles
Let's say I pull the same sequence Red, Blue, Blue, Red, but instead I have a multiple jars and I don't know which one I pulled from. 

The jar could have 70 red and 30 blue. Or it could have 30 blue and 70 red. Or it could have 50 red and 50 blue. What is the probability that I pulled the sequence? What is the probability of each jar being the one I pulled from?

# Viterbi Algorithm

Now lets use an example that is a bit more dynamic. Lets say that each time you pulled the marble, the jar either chagned or stayed the same. You might be asking when would this happen? Let's give some more realism to the example.

Let's say we are a campaign team for an election. Your budget can only afford a poll that can reveal if red or blue is in the lead for the election.

You know based on an expert's advice that there are 5 possible states

A. There is a 99% chance blue will be in the lead

B. There is a 75% chance blue will be in the lead

C. There is a 50% chance blue will be in the lead

D. There is a 25% chance blue will be in the lead

E. There is a 1% chance blue will be in the lead

To summarize, at any given point in time, we will know the history of observed values and their probabilities given a state. We also know the transition probabilities between states

We call these observed values the emission probabilities and we will note it with the symbol $\vec{y}$ where each $\vec{y}_t$ gives us the emmision spectrum at time point t.

What we want to do is figure out the state based on our information. To do that we must estimate the probability of every single state $\vec{s}_t$ based on the emmision. That is we want to create an a posteriori (after the fact) estimation of $P(\vec{s}_t | \vec{y}_t)$




## Math
So we know $\vec{y}_t$ and we need to calculate the most likely $\vec{s}_t$. That means we need to maximize $P(\vec{s}_t | \vec{y}_t)$ with respect to $\vec{s}_t$
$$\begin{align*}
P(\vec{s}_t | \vec{y}_t) &\propto
P(\vec{y}_t | \vec{s}_t) P( \vec{s}_t)\\
&= \prod_{t=1}^T P(y_t | s_t)P( \vec{s}_t)
\end{align*}
$$

So this is kindof tricky becasue we need $P( \vec{s}_t)$. We can use the rule of conditional probability

$$
\begin{align*}
P( \vec{s}_t) &= P(s_1,s_2,s_3,...,s_{t-1},s_{t}) \\
&= P(s_{t}|s_1,s_2,s_3,...,s_{t-1})P(s_1,s_2,s_3,...,s_{t-1}) \\
\end{align*}
$$

But we don't need all previous states. We only care about the relationships between $t-1$ and $t$ so $P(s_{t}|s_{t-1})P(s_1,s_2,s_3,...,s_{t-1}) = P(s_{t}|s_{t-1})P(s_{t-1})$
$$
\begin{align*}
P( \vec{s}_t) &= P(s_1,s_2,s_3,...,s_{t-1},s_{t}) \\
&= P(s_{t}|s_1,s_2,s_3,...,s_{t-1})P(s_1,s_2,s_3,...,s_{t-1}) \\
&= P(s_{t}|s_{t-1})P(s_{t-1}|s_{t-2})P(s_1,s_2,s_3,...,s_{t-2}) \\
\dots\\
&= P(s_{t}|s_{t-1})P(s_{t-1}|s_{t-2})...P(s_3|s_2)P(s_2|s_1)P(s_1)\\
&= P(s_1) \prod_{t=2}^T P(s_{t}|s_{t-1})
\end{align*}
$$
Putting that all together in our original equation we get

$$P(\vec{s}_t | \vec{y}_t) \propto \prod_{t=1}^t P(y_t | s_t) P(s_1) \prod_{t=2}^T P(s_{t}|s_{t-1})$$
But we aren't good at optimizing over products, so we take the log of each side. Log is an order preserving homomorphism, so if we maximize the log, the same value should maximize the original.

$$log \left[ 
    P(\vec{s}_t | \vec{y}_t)
    \right] \propto \sum_{t=1}^t 
    log \left[ P(y_t | s_t)
    \right] +
    log \left[ P(s_1)
    \right] +
    \sum_{t=2}^T log \left[ P(s_{t}|s_{t-1})
    \right] 
$$
     
Now we can just use the bellman equation to back propogate and find the most likely outcome




In [3]:
import numpy as np
import random
from collections import namedtuple
# COVID EXAMPLE

In [6]:
transitions = np.matrix([[0.99, 0.01],
                        [0.2, 0.8]])
emission_probs = np.array([[0.75,0.25],
                           [0.005, 0.995]])
cache = {}
State = namedtuple('State', ['time', 'state'])

prior = [0.5,0.5]

def immediate_value(state, action):
  if state.time < 0:
    return np.log(prior[action])
  emission = emissions[state.time]
  if state.time == T-1:
    return np.log(emission_probs[state.state, emission])
  return np.log(emission_probs[state.state, emission]) + np.log(transitions[state.state, action])

def possible_actions(state):
  if state.time < 0:
    return [i for i in range(n_states) if prior[i] > 0]
  else:
    return [i for i in range(n_states) if transitions[state.state, i] > 0]

def next_state(state, action):
  return State(state.time + 1, action)

def bellman(state):
  if state.time >= T:
    return (0, '')
  if state not in cache:
    cache[state] = max((immediate_value(state,action) + bellman(next_state(state, action))[0], action) for action in possible_actions(state))
  return cache[state]